In [1]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('D:/My Document/Khóa Luận Tốt Nghiệp/Model Reposity/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data, transform_relations

Current working folder: d:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA\notebooks
Current working folder: D:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA


In [2]:
llm_instance = LLM_Model()
relations = list(load_json_data('datasets/icews18/relation2id.json').keys())
output_dir = 'result/icews18_1/stage_1'

Loading data from config/llm_config.json
Loading data from datasets/icews18/relation2id.json


In [3]:
relations

['Make_statement',
 'Make_an_appeal_or_request',
 'Use_conventional_military_force',
 'Rally_opposition_against',
 'Demand',
 'Arrest,_detain,_or_charge_with_legal_action',
 'Criticize_or_denounce',
 'Engage_in_symbolic_act',
 'Praise_or_endorse',
 'Expel_or_deport_individuals',
 'Consider_policy_option',
 'Appeal_for_change_in_leadership',
 'Express_intent_to_engage_in_diplomatic_cooperation_(such_as_policy_support)',
 'Protest_violently,_riot',
 'Use_tactics_of_violent_repression',
 'Express_intent_to_meet_or_negotiate',
 'Appeal_for_policy_change',
 'Engage_in_diplomatic_cooperation',
 'Appeal_for_de-escalation_of_military_engagement',
 'Demonstrate_or_rally',
 'Accuse',
 'Demonstrate_military_or_police_power',
 'Threaten',
 'Share_intelligence_or_information',
 'Expel_or_withdraw',
 'Sign_formal_agreement',
 'Express_intent_to_settle_dispute',
 'Appeal_for_diplomatic_cooperation_(such_as_policy_support)',
 'Use_unconventional_violence',
 'Investigate',
 'Ease_administrative_sanctio

In [4]:
transform_relations(relations, llm_instance, output_dir)

Data has been converted to JSON and saved to result/icews18_1/stage_1\transformed_relations.json


dict_values(['Made a statement', 'Made an appeal or request', 'Used conventional military force', 'Rallied opposition against', 'Demanded', 'Arrested, detained, or charged with legal action', 'Criticized or denounced', 'Engaged in a symbolic act', 'Praised or endorsed', 'Expelled or deported individuals', 'Considered a policy option', 'Appealed for a change in leadership', 'Expressed intent to engage in diplomatic cooperation such as policy support', 'Protested violently and rioted', 'Used tactics of violent repression', 'Expressed intent to meet or negotiate', 'Appealed for policy change', 'Engaged in diplomatic cooperation', 'Appealed for de-escalation of military engagement', 'Demonstrated or rallied', 'Accused', 'Demonstrated military or police power', 'Threatened', 'Shared intelligence or information', 'Expelled or withdrew', 'Signed a formal agreement', 'Expressed intent to settle the dispute', 'Appealed for diplomatic cooperation such as policy support', 'Used unconventional vio